In [16]:
import PIL.Image
import numpy as np
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import imageio

from diffusers import UNet2DModel, DDIMScheduler, VQModel
import torch
from torch.utils.data import DataLoader
from torch.utils.checkpoint import checkpoint
import torch.nn.functional as F

from data.dataset import ImageDataset, CelebHQAttrDataset
from init_classifier import Classifier


class CheckpointedUNetWrapper(torch.nn.Module):
    def __init__(self, model):
        super(CheckpointedUNetWrapper, self).__init__()
        self.model = model

    def checkpointed_forward(self, module, *inputs):
        def custom_forward(*inputs):
            return module(*inputs)
        return checkpoint(custom_forward, *inputs)

    def forward(self, sample, timestep):

        # 1. time
        timesteps = timestep
        if not torch.is_tensor(timesteps):
            timesteps = torch.tensor([timesteps], dtype=torch.long, device=sample.device)
        elif torch.is_tensor(timesteps) and len(timesteps.shape) == 0:
            timesteps = timesteps[None].to(sample.device)

        # broadcast to batch dimension in a way that's compatible with ONNX/Core ML
        timesteps = timesteps * torch.ones(sample.shape[0], dtype=timesteps.dtype, device=timesteps.device)

        t_emb = self.model.time_proj(timesteps)
        #t_emb = t_emb.to(dtype=self.dtype)
        emb = self.model.time_embedding(t_emb)

        # 2. pre-process
        skip_sample = sample
        sample = self.model.conv_in(sample)

        # 3. down
        down_block_res_samples = (sample,)
        for downsample_block in self.model.down_blocks:
            if hasattr(downsample_block, "skip_conv"):
                sample, res_samples, skip_sample = self.checkpointed_forward(downsample_block, sample, emb, skip_sample)
                #sample, res_samples, skip_sample = downsample_block(
                #    hidden_states=sample, temb=emb, skip_sample=skip_sample
                #)
            else:
                #sample, res_samples = downsample_block(hidden_states=sample, temb=emb)
                sample, res_samples = self.checkpointed_forward(downsample_block, sample, emb)

            down_block_res_samples += res_samples
        
        # 4. mid
        sample = self.checkpointed_forward(self.model.mid_block, sample, emb)

        # 5. up
        skip_sample = None
        for upsample_block in self.model.up_blocks:
            res_samples = down_block_res_samples[-len(upsample_block.resnets) :]
            down_block_res_samples = down_block_res_samples[: -len(upsample_block.resnets)]

            if hasattr(upsample_block, "skip_conv"):
                #sample, skip_sample = upsample_block(sample, res_samples, emb, skip_sample)
            else:
                #sample = upsample_block(sample, res_samples, emb)








        sample = self.model.conv_in(sample)

        # Apply checkpointing to down blocks
        for block in self.model.down_blocks:
            sample = self.checkpointed_forward(block, sample, timestep)
        
        # Apply checkpointing to middle block
        sample = self.checkpointed_forward(self.model.mid_block, sample, timestep)

        # Apply checkpointing to up blocks
        for block in self.model.up_blocks:
            sample = self.checkpointed_forward(block, sample, timestep)
        
        # Apply conv_out layer
        sample = self.model.conv_out(sample)
        
        return {"sample": sample}


def color_loss(images, target_color=(1.0, 0.0, 0.0)):
    """Given a target color (R, G, B) return a loss for how far away on average
    the images' pixels are from that color. Defaults to a light teal: (0.1, 0.9, 0.5)"""
    target = torch.tensor(target_color).to(images.device) * 2 - 1  # Map target color to (-1, 1)
    target = target[None, :, None, None]  # Get shape right to work with the images (b, c, h, w)
    error = torch.abs(images - target).mean()  # Mean absolute difference between the image pixels and the target color
    return error

def classifier_loss(classifier, images, targets, idx):
    preds = classifier(images)
    if idx % 10 == 0:
        print(f"Classifier prediction: {preds[0][31]}")
    error = torch.abs(preds[0][31] - targets).mean()
    return error

def minDist_loss(images, original_images):
    error = torch.abs(images - original_images).mean()
    return error

#enable checkpointing


# data loading with ground truth no smiling
data = ImageDataset('/home/dai/GPU-Student-2/Cederic/DataSciPro/data/misclsData_gt1', image_size=256, exts=['jpg', 'JPG', 'png'], do_augment=False, sort_names=True)
dataloader = DataLoader(data, batch_size=1, shuffle=False)


#
device = "cuda" if torch.cuda.is_available() else "cpu"
model_id = "CompVis/ldm-celebahq-256"
seed = 3

# load all models
classifier = Classifier.load_from_checkpoint("/home/dai/GPU-Student-2/Cederic/pjds_group8/cls_checkpoints/ffhq256.2024-05-26 18:46:33.ckpt",
                                             input_dim = data[0]['img'].shape,
                                             num_classes = len(CelebHQAttrDataset.id_to_cls))
classifier.to(device)
classifier.eval()
# check functionality of classifier
all_outputs = []
with torch.no_grad():
    for batch in dataloader:
        inputs = batch['img'].to(classifier.device)
        outputs = classifier(inputs)
        print(outputs[0][31])

        preds_binary = torch.sigmoid(outputs[:, CelebHQAttrDataset.cls_to_id['Smiling']].cpu()) > 0.5
        all_outputs.append(preds_binary) 
all_outputs = torch.cat(all_outputs, dim=0)
print(all_outputs)

unet = UNet2DModel.from_pretrained("CompVis/ldm-celebahq-256", subfolder="unet")
vqvae = VQModel.from_pretrained("CompVis/ldm-celebahq-256", subfolder="vqvae")
scheduler = DDIMScheduler.from_config("CompVis/ldm-celebahq-256", subfolder="scheduler")
scheduler.set_timesteps(num_inference_steps=25)

unet.to(device)
vqvae.to(device)

checkpointed_unet = CheckpointedUNetWrapper(unet)

## Inversion
def invert(
    start_latents,
    guidance_scale=3.5,
    num_inference_steps=50,
    device=device,
):

    # Latents are now the specified start latents
    latents = start_latents.clone()

    # We'll keep a list of the inverted latents as the process goes on
    intermediate_latents = []

    # Set num inference steps
    scheduler.set_timesteps(num_inference_steps, device=device)

    # Reversed timesteps <<<<<<<<<<<<<<<<<<<<
    timesteps = reversed(scheduler.timesteps)

    for i in tqdm(range(1, num_inference_steps), total=num_inference_steps - 1):

        # We'll skip the final iteration
        if i >= num_inference_steps - 1:
            continue

        t = timesteps[i]

        # Expand the latents if we are doing classifier free guidance
        latent_model_input = latents
        latent_model_input = scheduler.scale_model_input(latent_model_input, t)

        # Predict the noise residual
        noise_pred = checkpointed_unet(latent_model_input, t).sample

        current_t = max(0, t.item() - (1000 // num_inference_steps))  # t
        next_t = t  # min(999, t.item() + (1000//num_inference_steps)) # t+1
        alpha_t = scheduler.alphas_cumprod[current_t]
        alpha_t_next = scheduler.alphas_cumprod[next_t]

        # Inverted update step (re-arranging the update step to get x(t) (new latents) as a function of x(t-1) (current latents)
        latents = (latents - (1 - alpha_t).sqrt() * noise_pred) * (alpha_t_next.sqrt() / alpha_t.sqrt()) + (
            1 - alpha_t_next
        ).sqrt() * noise_pred

        # Store
        intermediate_latents.append(latents)

    return torch.cat(intermediate_latents)


class LatentNoise(torch.nn.Module):
    """
    The LatentNoise Module makes it easier to update the noise tensor with torch optimizers.
    """

    def __init__(self, noise: torch.Tensor):
        super().__init__()
        self.noise = torch.nn.Parameter(noise)

    def forward(self):
        return self.noise


def diffusion_pipe(noise_module: LatentNoise, num_inference_steps):
        z = noise_module()
        for i in range(start_step, num_inference_steps):
            t = scheduler.timesteps[i]
            z = scheduler.scale_model_input(z, t)
            with torch.no_grad():
                noise_pred = unet(z, t)["sample"]
            z = scheduler.step(noise_pred, t, z).prev_sample
        return z

def plot_to_pil(tensor):
    image = tensor.cpu().permute(0, 2, 3, 1).clip(-1,1) * 0.5 + 0.5
    image = PIL.Image.fromarray(np.array(image[0].detach().numpy() * 255).astype(np.uint8))
    plt.imshow(image)
    plt.axis('off')
    plt.show()

def tensor_to_pil_image(tensor):
    image = tensor.cpu().permute(0, 2, 3, 1).clip(-1,1) * 0.5 + 0.5
    image = PIL.Image.fromarray(np.array(image[0].detach().numpy() * 255).astype(np.uint8))
    return image


# conditional sampling
guidance_cls = 2
guidance_dist = 15
num_inference_steps = 50
num_optimization_steps = 50
start_step = 10


for step, batch in tqdm(enumerate(dataloader), total=len(dataloader)):
    with torch.no_grad():
        z = vqvae.encode(batch['img'].to(device))   # encode the image in the latent space
    z = z.latents
    
    inverted_latents = invert(z, num_inference_steps=50)                  # do the ddim scheduler reversed to add noise to the latents
    z = inverted_latents[-(start_step + 1)].unsqueeze(0)                  # use these latents to start the sampling. better performance when using not the last latent sample
    noise_module = LatentNoise(z.clone()).to(device)                    # convert latent noise to a parameter module for optimization
    noise_module.noise.requires_grad = True
    intermediate_results = []   # list to store the results of the steering

    #new_z = z.clone().detach().requires_grad_(True)

    """
    with torch.no_grad():
        im = vqvae.decode(inverted_latents[-1].unsqueeze(0))
        im_processed = im[0].cpu().permute(0, 2, 3, 1).clip(-1,1) * 0.5 + 0.5
        im_pil = PIL.Image.fromarray(np.array(im_processed[0] * 255).astype(np.uint8))
        plt.imshow(im_pil)
        plt.axis('off')
        plt.show()
    """
    
    optimizer = torch.optim.Adam(
        noise_module.parameters(), lr=0.01, maximize=False # not minimize gradient ascent
    )
    
    """
    
    for i in tqdm(range(start_step, num_inference_steps)):

        t = scheduler.timesteps[i]

        # Prepare the model input
        model_input = scheduler.scale_model_input(x, t)

        # predict noise residual of previous image
        with torch.no_grad():
            noise_pred = unet(model_input, t)["sample"]

        # Set x.requires_grad to True
        x = x.detach().requires_grad_()

        # Get the predicted x0
        x0 = scheduler.step(noise_pred, t, x).pred_original_sample
        decoded_x0 = vqvae.decode(x0)[0]
        decoded_x = vqvae.decode(x)[0]

        # Calculate loss
        #loss = color_loss(x0) * guidance_loss_scale
        loss = guidance_cls * classifier_loss(classifier, decoded_x0, 1.0) + guidance_dist * minDist_loss(decoded_x0, decoded_x)
        if i % 10 == 0:
            print(i, "loss:", loss.item())

        # Get gradient
        cond_grad = -torch.autograd.grad(loss, x)[0]

        # Modify x based on this gradient
        x = x.detach() + cond_grad 

        # Now step with scheduler
        x = scheduler.step(noise_pred, t, x).prev_sample

    # decode image with vae
    with torch.no_grad():
        image = vqvae.decode(x)[0]
    """
    x = torch.zeros_like(z)
    for i in tqdm(range(num_optimization_steps)):
            optimizer.zero_grad()
            x = diffusion_pipe(noise_module, num_inference_steps) # forward
            decoded_x = vqvae.decode(x)[0]

            if i % 10 == 0:
                intermediate_results.append(decoded_x)
            #    plot_to_pil(decoded_x)

            loss = classifier_loss(classifier, decoded_x, 1.0, i) # apply losses
            if i % 10 == 0:
                print(i, "loss:", loss.item())
            loss.backward()
            optimizer.step()

    with torch.no_grad():
        image = vqvae.decode(x)[0]
    
    images = [tensor_to_pil_image(tensor) for tensor in intermediate_results]
    gif_path = "/home/dai/GPU-Student-2/Cederic/DataSciPro/gifs/output.gif"
    imageio.mimsave(gif_path, images, format='GIF', duration=1.0, loop=0)  # duration is in seconds


    # process image
    image_processed = image.cpu().permute(0, 2, 3, 1).clip(-1,1) * 0.5 + 0.5
    image_pil = PIL.Image.fromarray(np.array(image_processed[0] * 255).astype(np.uint8))
    ori_processed = batch['img'].cpu().permute(0, 2, 3, 1).clip(-1,1) * 0.5 + 0.5
    ori_image = PIL.Image.fromarray(np.array(ori_processed[0] * 255).astype(np.uint8))

    fig, axs = plt.subplots(1, 2, figsize=(10, 5))
    axs[0].imshow(image_pil)
    axs[0].axis('off')
    axs[1].imshow(ori_image)
    axs[1].axis('off')
    plt.show()

    image_pil.save(f"generated_image_{seed}.png")
    print('finish')

tensor(-1.2285, device='cuda:0')
tensor([False])


/home/dai/GPU-Student-2/Cederic/anaconda3/envs/diffcounter/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/dai/GPU-Student-2/Cederic/anaconda3/envs/diffcounter/lib/python3.10/site-packages/diffusers/configuration_utils.py:244: FutureWarning: It is deprecated to pass a pretrained model name or path to `from_config`.If you were trying to load a scheduler, please use <class 'diffusers.schedulers.scheduling_ddim.DDIMScheduler'>.from_pretrained(...) instead. Otherwise, please make sure to pass a configuration dictionary instead. This functionality will be removed in v1.0.0.
  deprecate("config-passed-as-path", "1.0.0", deprecation_message, standard_warn=False)
The config attributes {'timestep_values': None, 'timesteps': 1000} were passed to DDIMScheduler, but are 

RuntimeError: Given groups=1, weight of size [224, 3, 3, 3], expected input[1, 896, 8, 8] to have 3 channels, but got 896 channels instead